### Train the Model and get the Accuracy and Loss curves - On Generated Data , see the usage of fit_generator() method.

### Create new images and try the same model to be trained


In [2]:
from keras import backend as k
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers.convolutional import Convolution2D, Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.datasets import mnist,cifar10
from keras.optimizers import RMSprop, Adam, SGD
import numpy as np
import matplotlib.pyplot as plt
from keras.utils import np_utils
from keras.models import Sequential
%matplotlib inline

In [3]:
from keras.preprocessing.image import ImageDataGenerator
from keras.datasets import cifar10
import numpy as np
NUM_TO_AUGMENT = 5

# Repititive steps as last time for training the model

OPTIMIZER = RMSprop()
NB_EPOCHS = 50   # decreasing the number of epochs to reduce the time for MLP, also for the RMSPROP and ADAM optimizers
SPLIT_PARAM = 0.2
NB_CLASSES = 10
VERBOSE =1
BATCH_SIZE = 128
DROPOUT_RATE = 0.25
NB_ROW, NB_COLS = 32, 32
NB_CHANNELS = 3
INPUT_SHAPE = (NB_ROW,NB_COLS, NB_CHANNELS)

# load dataset
(X_train, Y_train), (X_test, Y_test) = cifar10.load_data()

Y_train = np_utils.to_categorical(Y_train, NB_CLASSES)
Y_test = np_utils.to_categorical(Y_test, NB_CLASSES)

# augumenting
print("Augumenting training set images ... ")
datagen = ImageDataGenerator(
rotation_range=40,
width_shift_range=0.2,
height_shift_range=0.2,
zoom_range=0.2,
horizontal_flip=True,
fill_mode='nearest')

Augumenting training set images ... 


### Train the Model and get the Accuracy and Loss curves - On Generated Data , see the usage of fit_generator() method.

In [ ]:
xtas, ytas = [], []
for i in range(X_train.shape[0]):
    num_aug = 0
    x = X_train[i] # (3, 32, 32)
    x = x.reshape((1,) + x.shape) # (1, 3, 32, 32)
    for x_aug in datagen.flow(x, batch_size=1,
    save_to_dir='/Users/tkmacl9/Desktop/FastAIDLCourse/nbs/KerasDeepLearning/preview', save_prefix='cifar', save_format='jpeg'):
    if num_aug >= NUM_TO_AUGMENT:
        break
    xtas.append(x_aug[0])
    num_aug += 1

In [4]:
model = Sequential()
# 1st layer of the DNN
model.add(Conv2D(32, (3,3), padding='same', input_shape=INPUT_SHAPE))
model.add(Activation('relu'))
model.add(Conv2D(32, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(DROPOUT_RATE))

# 2nd layer of the DNN
model.add(Conv2D(64, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(DROPOUT_RATE))

# Final dense layer
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(DROPOUT_RATE))
model.add(Dense(NB_CLASSES))
model.add(Activation('softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 64)        18496     
__________

In [5]:
datagen

In [6]:
#fit the dataget
datagen.fit(X_train)

In [7]:
# train
model.compile(optimizer=OPTIMIZER, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# score = model.evaluate(X_test, Y_test,
# batch_size=BATCH_SIZE, verbose=VERBOSE)
# print("Test score:", score[0])
# print('Test accuracy:', score[1])

In [9]:
history = model.fit_generator(datagen.flow(X_train, Y_train, 
                               batch_size=BATCH_SIZE) ,samples_per_epoch=X_train.shape[0] ,epochs=NB_EPOCHS, 
                              verbose=VERBOSE)

Epoch 1/50


/Users/tkmacl9/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., verbose=1, epochs=50, steps_per_epoch=1562)`
  app.launch_new_instance()


ValueError: Error when checking target: expected activation_6 to have shape (None, 1) but got array with shape (32, 10)